In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
from selenium import webdriver

# Book.toscrape

In [16]:
name=[]
#kitab saytindan melumat cekilmesi
r=requests.get("https://books.toscrape.com/")
soup=BeautifulSoup(r.content,"html.parser")
a=soup.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
kitablar=[i.get_text().strip("\n").split("\n") for i in a]
stok=[]
qiymet=[]
for c in kitablar:
    qiymet.append(c[2])#qiymet
    stok.append(c[6])#stokdadi ya yox

a=soup.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
for i in a:
    linkler=i.find_all("div",class_="image_container")
    for link in linkler:
        k=link.find_all("a")
        
        for link_s in k:
            linksonu=link_s.get("href")
            linkbasi="https://books.toscrape.com/"
            link_tamami=linkbasi+linksonu
#             print(link_tamami)
            
           
            detay_r=requests.get(link_tamami)
            detay_soup=BeautifulSoup(detay_r.content,"html.parser")
            
            detay_=detay_soup.find_all("div",class_="col-sm-6 product_main")
            for book_name in detay_:
                ad=book_name.find("h1").get_text()
                name.append(ad)
                
            
        
dictionary={"ad":name,"qiymet":qiymet,"stok":stok}
data=pd.DataFrame(dictionary)   
    

# IMDB

In [6]:
r=requests.get("https://www.imdb.com/chart/top/?ref_=nv_mv_250")
soup=BeautifulSoup(r.content,"html.parser")
filimler=soup.find_all("td",class_="titleColumn")
filim_adi=[]
filim_reytingi=[]
filim_ili=[]
for i in filimler:
    filim_adi.append(i.find("a").get_text())
    filim_ili.append(i.find("span",class_="secondaryInfo").get_text())


soup_rayting=soup.find_all("tr")
for i in soup_rayting:
    reytingler=i.find_all(class_="ratingColumn imdbRating")
    for j in reytingler:
        filim_reytingi.append(j.find("strong").get_text())

    
film_cedveli={"Filim_adi":filim_adi,"Filim_il":filim_ili,"Filim_reytingi":filim_reytingi}
cedvel=pd.DataFrame(film_cedveli)
cedvel.to_csv("filmler.csv")

# Turbo.az

In [ ]:
sehife=1
liste=[]
qiymet=[]
while sehife<2:  #seife sayini teyin etmek ucun istifade edilir
    print(sehife)
    r=requests.get("https://turbo.az/autos?page={}".format(sehife))
    soup=BeautifulSoup(r.content,"html.parser")
    linkler=soup.find_all("a",class_="products-i__link")
    header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
    for i in linkler:
        link_sonu=i.get("href")
        link_tamami="https://turbo.az/"+link_sonu
        response=requests.get(link_tamami,headers=header)
        soup=BeautifulSoup(response.content,"html.parser")
        qiymetler=soup.find_all("div",class_="product-price__i product-price__i--bold")
        for j in qiymetler:
            qiymet.append(j.get_text().strip("AZN USD"))

        detay=soup.find_all("div",class_="product-properties tz-d-flex tz-justify-between tz-gap-10")
        for k in detay:
            label=k.find_all("label",class_="product-properties__i-name")
            span=k.find_all("span",class_="product-properties__i-value")
            son={}
            for i in range(len(label)):
                son[label[i].text]=span[i].text
            liste.append(son)
    sehife+=1

#Sonda aldigimiz data Frame cevrilir ordanda csv fayl import edilir 
Frame=pd.DataFrame(liste)
Frame.insert(loc=len(Frame.columns),column="Qiymet",value=qiymet)
Frame["Qiymet"]=Frame["Qiymet"].str.strip("AZN USD")
Frame
        

    
    

    #     for k in elametler:
#         print(k.find("label",class_="product-properties__i-name").text,"--",k.find("span",class_="product-properties__i-value").text)
#     print("---------------------------")
        
    

1


# Kitabyurdu

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

kitablar=[]
qiymet=[]
yazici=[]
kitab_adi=[]
qiymetler=[]
re=requests.get("https://www.kitapyurdu.com/index.php?route=product/publisher_products/all&&publisher_id=7818&filter_in_stock=1&sort=purchased_365&order=DESC&limit=100")
soup=BeautifulSoup(re.content,"lxml")
kitab_linkleri=soup.find_all("a",class_="pr-img-link")
for i in kitab_linkleri:
    link=i.get("href")
    re=requests.get(link) 
    soup=BeautifulSoup(re.content,"lxml") 
    qiymet=soup.find("div",class_="price__item")
    yazicilar=soup.find("div",class_="pr_producers__manufacturer")
    kitab_adlari=soup.find("h1",class_="pr_header__heading")
    try:
        yazici.append(yazicilar.get_text().strip("\n"))
        kitab_adi.append(kitab_adlari.get_text().strip("\n"))
        qiymetler.append(qiymet.get_text())
    except:
        yazici.append("Namelum")
        kitab_adi.append("Namelum")
        qiymetler.append("Namelum")
    
    #Xususiyetler
    
    detaylar=soup.find_all("td")
    cedvel={}
    for i in range(0,len(detaylar),2):
        cedvel[detaylar[i].get_text()]=detaylar[i+1].text
    kitablar.append(cedvel)
        
a=pd.DataFrame(kitablar)
a.insert(loc=0,column="Kitap_ismi",value=kitab_adi)
a.insert(loc=1,column="Yazar",value=yazici)
a.insert(loc=9,column="Fiyat",value=qiymetler)
a.columns=a.columns.str.replace(":","")